# 연예인 이미지 크롤링

In [3]:
import cv2
print(cv2.__version__)          # 버전 출력
print(hasattr(cv2, 'face'))     # 얼굴 인식 모듈 확인 (True이면 성공)
import urllib.request
import os

4.11.0
True


In [4]:
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import requests
import numpy as np

In [5]:
filename = './data/image'
classifier = cv2.CascadeClassifier(filename)

In [6]:
# url = 'https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_frontalface_default.xml'
# save_path = 'haarcascade_frontalface_default.xml'

# urllib.request.urlretrieve(url, save_path)
# print("다운로드 완료!")

# [1] 얼굴 부분 검출

In [ ]:

# 1. 분류기 로드
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# 2. 이미지 불러오기
img = cv2.imread('./data/image/carina.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    
                                            
# 3. 얼굴 검출
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

# 4. 얼굴 영역 그리기
for (x, y, w, h) in faces: 
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

# 5. 결과 보기
cv2.imshow('Face Detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def cut_face (img_path, save_path):
	# 1. 얼굴 분류기 로드
	face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
	img = cv2.imread(img_path)
	gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

	# 3. 얼굴 검출
	faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

	# 4. 저장 경로 설정
	save_dir = './data/image/faces'
	os.makedirs(save_dir, exist_ok=True)

	# 5. 얼굴 잘라서 저장하기
	for i, (x, y, w, h) in enumerate(faces):
		face_img = img[y:y+h, x:x+w]  # 컬러 이미지에서 얼굴 부분만 자름
		save_path = os.path.join(save_dir, f"carina_face_{i}.jpg")
		cv2.imwrite(save_path, face_img)
		print(f"✅ 얼굴 저장 완료: {save_path}")

		# 원본 이미지에 박스 그리기
		cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)

	# 6. 결과 보기
	cv2.imshow('Detected Faces', img)
	cv2.waitKey(0)
	cv2.destroyAllWindows()

# [2] 흑백 이미지 증강 

In [9]:
def augment_images_in_folder(img_dir, save_dir):
    # 1. 저장 폴더 생성
    os.makedirs(save_dir, exist_ok=True)

    # 2. 폴더 내 이미지 반복 처리
    for filename in os.listdir(img_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            file_path = os.path.join(img_dir, filename)
            img = cv2.imread(file_path)

            if img is None:
                print(f"❌ 이미지 불러오기 실패: {file_path}")
                continue

            # 3. 전처리: 흑백 + 리사이즈
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (128, 128))

            # 4. 증강 이미지 생성
            augmented_images = [resized]                          # 원본
            augmented_images.append(cv2.flip(resized, 1))         # 좌우 반전

            for angle in [15, -15]:                               # 회전
                M = cv2.getRotationMatrix2D((64, 64), angle, 1.0)
                rotated = cv2.warpAffine(resized, M, (128, 128), borderMode=cv2.BORDER_REFLECT)
                augmented_images.append(rotated)

            # 5. 저장
            base_name = os.path.splitext(filename)[0]
            for idx, aug_img in enumerate(augmented_images):
                save_path = os.path.join(save_dir, f"{base_name}_aug{idx}.jpg")
                success = cv2.imwrite(save_path, aug_img)
                if success:
                    print(f"✅ 저장 완료: {save_path}")
                else:
                    print(f"❌ 저장 실패: {save_path}")

In [28]:

def cut_faces_in_folder(img_dir, save_dir):
    # 1. 얼굴 분류기 로드
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # 2. 저장 폴더 생성
    os.makedirs(save_dir, exist_ok=True)

    # 3. 이미지 폴더 내 파일 반복
    for filename in os.listdir(img_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            file_path = os.path.join(img_dir, filename)
            img = cv2.imread(file_path)
            if img is None:
                print(f"❌ 이미지 불러오기 실패: {file_path}")
                continue

            color_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            faces = face_cascade.detectMultiScale(color_img, scaleFactor=1.1, minNeighbors=5)

            print(f"[{filename}] 얼굴 수: {len(faces)}")

            for i, (x, y, w, h) in enumerate(faces):
                face_img = img[y:y+h, x:x+w]
                save_path = os.path.join(save_dir, f"{os.path.splitext(filename)[0]}_face_{i}.jpg")
                success = cv2.imwrite(save_path, face_img)
                if success:
                    print(f"✅ 얼굴 저장 완료: {save_path}")
                else:
                    print(f"❌ 저장 실패: {save_path}")

            # augment_and_save(face_img, save_path, i)
            # ▶️ 선택적으로 결과 이미지 보기 (원할 때만 사용)
            # cv2.imshow('Detected Faces', img)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()


In [31]:
# img_path = './data/winter_2'
# save_dir='./data/image/winter_face_2'
# cut_faces_in_folder(img_path,save_dir)

[winter_2(1).jpeg] 얼굴 수: 2
✅ 얼굴 저장 완료: ./data/image/winter_face_2\winter_2(1)_face_0.jpg
✅ 얼굴 저장 완료: ./data/image/winter_face_2\winter_2(1)_face_1.jpg
[winter_2(1).jpg] 얼굴 수: 3
✅ 얼굴 저장 완료: ./data/image/winter_face_2\winter_2(1)_face_0.jpg
✅ 얼굴 저장 완료: ./data/image/winter_face_2\winter_2(1)_face_1.jpg
✅ 얼굴 저장 완료: ./data/image/winter_face_2\winter_2(1)_face_2.jpg
[winter_2(10).jpg] 얼굴 수: 1
✅ 얼굴 저장 완료: ./data/image/winter_face_2\winter_2(10)_face_0.jpg
[winter_2(100).jpg] 얼굴 수: 1
✅ 얼굴 저장 완료: ./data/image/winter_face_2\winter_2(100)_face_0.jpg
[winter_2(101).jpg] 얼굴 수: 1
✅ 얼굴 저장 완료: ./data/image/winter_face_2\winter_2(101)_face_0.jpg
[winter_2(102).jpg] 얼굴 수: 2
✅ 얼굴 저장 완료: ./data/image/winter_face_2\winter_2(102)_face_0.jpg
✅ 얼굴 저장 완료: ./data/image/winter_face_2\winter_2(102)_face_1.jpg
[winter_2(103).jpg] 얼굴 수: 0
[winter_2(104).jpg] 얼굴 수: 1
✅ 얼굴 저장 완료: ./data/image/winter_face_2\winter_2(104)_face_0.jpg
[winter_2(105).jpg] 얼굴 수: 1
✅ 얼굴 저장 완료: ./data/image/winter_face_2\winter_2(105)_face_0

In [33]:
# img_dir ='./data/image/winter_face_2'
# save_dir = './data/image/gray_others'
# augment_images_in_folder(img_dir,save_dir)

✅ 저장 완료: ./data/image/gray_others\winter_2(1)_face_2_aug0.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(1)_face_2_aug1.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(1)_face_2_aug2.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(1)_face_2_aug3.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(10)_face_0_aug0.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(10)_face_0_aug1.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(10)_face_0_aug2.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(10)_face_0_aug3.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(100)_face_0_aug0.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(100)_face_0_aug1.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(100)_face_0_aug2.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(100)_face_0_aug3.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(101)_face_0_aug0.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(101)_face_0_aug1.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(101)_face_0_aug2.jpg
✅ 저장 완료: ./data/image/gray_others\winter_2(101)_face